In [1]:
import keras as ks
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, Lambda
from keras.models import Model, Sequential
from keras.initializers import random_normal, zeros
from keras.regularizers import l2
from keras.utils.np_utils import to_categorical
from keras.utils import Sequence
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10

Using TensorFlow backend.


In [2]:
def VGG16_train(include_top=True, weights='vggface',
          input_tensor=None, input_shape=None,
          pooling=None,weights_path=None,
          classes=8):
    
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32,32,3)))
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 2
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 3
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Block 4
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


    # Block 5
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #fc layer
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
            
    return model

In [3]:
vgg_model = VGG16_train(classes=10)

vgg_model.compile(optimizer=SGD(lr=0.01, momentum=0.9, decay=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 256)         295168    
__________

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
def preprocess_input(x_temp, data_format=None, version=1):
    if data_format is None:
        data_format = K.image_data_format()
    assert data_format in {'channels_last', 'channels_first'}

    if version == 1:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 93.5940
            x_temp[:, 1, :, :] -= 104.7624
            x_temp[:, 2, :, :] -= 129.1863
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 93.5940
            x_temp[..., 1] -= 104.7624
            x_temp[..., 2] -= 129.1863

    elif version == 2:
        if data_format == 'channels_first':
            x_temp = x_temp[:, ::-1, ...]
            x_temp[:, 0, :, :] -= 91.4953
            x_temp[:, 1, :, :] -= 103.8827
            x_temp[:, 2, :, :] -= 131.0912
        else:
            x_temp = x_temp[..., ::-1]
            x_temp[..., 0] -= 91.4953
            x_temp[..., 1] -= 103.8827
            x_temp[..., 2] -= 131.0912
    else:
        raise NotImplementedError

    return x_temp

In [7]:
batch_size = 32
num_classes = 10
epochs = 50

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

X_train = preprocess_input(x_train)
X_test = preprocess_input(x_test)

y_train = ks.utils.to_categorical(y_train, num_classes)
y_test = ks.utils.to_categorical(y_test, num_classes)

50000 train samples
10000 test samples


In [8]:
log_dir = "./models/log/"

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=0.00001)

checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

num_train = X_train.shape[0]
num_val = X_test.shape[0]

In [8]:
#vgg_model.fit(x=X_train, y=y_train, batch_size=batch_size, epochs=epochs,
#      validation_data=(X_test, y_test), callbacks=[reduce_lr])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 74s 1ms/step - loss: 1.8280 - acc: 0.2894 - val_loss: 1.4245 - val_acc: 0.4673
Epoch 2/50
50000/50000 [==============================] - 68s 1ms/step - loss: 1.2163 - acc: 0.5598 - val_loss: 1.1128 - val_acc: 0.6078
Epoch 3/50
50000/50000 [==============================] - 67s 1ms/step - loss: 0.8819 - acc: 0.6939 - val_loss: 0.8465 - val_acc: 0.7106
Epoch 4/50
50000/50000 [==============================] - 68s 1ms/step - loss: 0.6730 - acc: 0.7715 - val_loss: 0.7818 - val_acc: 0.7420
Epoch 5/50
50000/50000 [==============================] - 68s 1ms/step - loss: 0.5113 - acc: 0.8261 - val_loss: 0.6977 - val_acc: 0.7711
Epoch 6/50
50000/50000 [==============================] - 67s 1ms/step - loss: 0.3815 - acc: 0.8701 - val_loss: 0.6560 - val_acc: 0.7914
Epoch 7/50
50000/50000 [==============================] - 67s 1ms/step - loss: 0.2697 - acc: 0.9098 - val_loss: 0.7415 - val_acc

In [ ]:
vgg_model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=datagen.flow(X_test, y_test, batch_size=batch_size),
    validation_steps=max(1, num_val//batch_size),
    epochs=epochs,
    initial_epoch=0,
    callbacks=[checkpoint, reduce_lr, early_stopping])

Epoch 1/50
1562/1562 [==============================] - 75s 48ms/step - loss: 1.9792 - acc: 0.2279 - val_loss: 1.8881 - val_acc: 0.2741
Epoch 2/50
1562/1562 [==============================] - 68s 44ms/step - loss: 1.6230 - acc: 0.3803 - val_loss: 1.4806 - val_acc: 0.4471
Epoch 3/50
1562/1562 [==============================] - 69s 44ms/step - loss: 1.3865 - acc: 0.4932 - val_loss: 1.2541 - val_acc: 0.5428
Epoch 4/50
1562/1562 [==============================] - 69s 44ms/step - loss: 1.1953 - acc: 0.5733 - val_loss: 1.1152 - val_acc: 0.6068
Epoch 5/50
1562/1562 [==============================] - 69s 44ms/step - loss: 1.0645 - acc: 0.6227 - val_loss: 1.0437 - val_acc: 0.6286
Epoch 6/50
1562/1562 [==============================] - 70s 45ms/step - loss: 0.9611 - acc: 0.6627 - val_loss: 0.9845 - val_acc: 0.6614
Epoch 7/50
1562/1562 [==============================] - 71s 46ms/step - loss: 0.8885 - acc: 0.6892 - val_loss: 0.9328 - val_acc: 0.6726
Epoch 8/50
  42/1562 [..........................